In [ ]:
from pymongo import MongoClient

def get_mongo_database(db_name, host='localhost', port=27017, username=None, password=None):
    
    """ Get named database from MongoDB with/out authentification """
    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s/%s' % (username, password, host, db_name)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)
        
    return conn[db_name]

In [ ]:

# Connect to db using dataset:
import dataset
import datafreeze

db = dataset.connect('mssql+pyodbc://ORLEBIDEVDB/INTEGRATION?driver=SQL+Server+Native+Client+11.0')

# Querying:
mktJul2018Donors = db.query(
"""
select
    mkt.person_id,
    dd.FullDateUSA,
    mkt.min_reg_id,
    mkt.total_platelet_donations
from
(
    select MKT.personid PERSON_ID, MIN(mkt.COLLECTIONDATESK) MIN_REG_DATE,MIN(registrationid) MIN_REG_ID,COUNT(1) total_platelet_donations
    from [Integration].[dbo].[INT_MKTCollectionDetails] mkt
    where mkt.DonationTypeSK in (2,5,7,26)		
     and mkt.CompletedFlag >= 8		
     and upper(mkt.GENDER) in ('M','F')		
     and (MKT.collectiondatesk >= '20180701' and MKT.collectiondatesk < '20180801')
    --and reg.person_id = 2237761
    group by MKT.personid
) mkt
JOIN [Integration].[dbo].[VW_INT_DIMDATE] dd
    ON dd.dateKey = mkt.MIN_REG_DATE
""")

# coll.insert_many(mktJul2018Donors)          

In [11]:
DB_REGS_JUL18 = 'regsJul18'
COLL_PLDONSJUL18 = 'plDonorsJul18'

db = get_mongo_database(DB_REGS_JUL18)
coll = db[COLL_PLDONSJUL18]

# get keys in the collection
document = coll.find_one()

for k in document:
    print(k)


resPID = coll.find({'person_id': 50939043})    
list(resPID)

_id
person_id
FullDateUSA
min_reg_id
total_platelet_donations


[{'_id': ObjectId('5b81c449cb27571e16b36405'),
  'person_id': 50939043,
  'FullDateUSA': '07/05/2018',
  'min_reg_id': 108539841,
  'total_platelet_donations': 1}]

In [ ]:
# Find all of the persons with more than 3 platelet donations:
res_highPL = coll.find({'total_platelet_donations': {'$gt': 3}})
list(res_highPL)

In [ ]:
# Find all of the persons with more than 3 platelet donations or dontated on 07/31/2018
res_highPL_MaxDate = coll.find({'$or': [ {'total_platelet_donations': {'$gt': 3} }, {'FullDateUSA': '07/31/2018'} ] })

list(res_highPL_MaxDate)

In [ ]:
# Turn the collection back into a Python list of dictionaries:
def mongo_coll_to_dict(dbname='test', collname='test', query={}, del_id=True, **kw):
    
    db = get_mongo_database(dbname, **kw)
    res = list(db[collname].find(query))
    
    if del_id:
        # Go through the results in res:
        for r in res:
            # remove Mongodb id value added to our import:
            r.pop('_id')
            
    return res

# Create a python dictionary from MongoDB data:
mongo_coll_to_dict(DB_REGS_JUL18, COLL_PLDONSJUL18)